In [1]:
import scipy
import scipy.stats as sps
import scipy.integrate as integr
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
def Chi2XtoInf(x, k):
    return scipy.stats.chi2(k).sf(x)

In [3]:
def Count(sample):
    c = Counter(sample)
    return c
def Heights(counts, n):
    cur_h = 0
    heights = dict()
    for cur_key in counts:
        frequence = counts[cur_key] / n
        heights[cur_key] = cur_h + frequence
        cur_h += frequence
        
    return heights
nums = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
sample = []
for i in range(len(nums)):
    tmp = [i] * nums[i]
    sample += tmp
print(sample)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9]


## a) Проверка c помощью χ²

### p-value для χ²

In [4]:
DeltaChi2 = 0
for i in range(len(nums)):
    DeltaChi2 += 0.1 * (nums[i] - 10) ** 2
p_value1 = Chi2XtoInf(DeltaChi2, 9)
p_value1

0.058984030544419454

## Проверка по Колмогорову

In [5]:
def KolmogorovDelta(args, sample, n, func):
    h_last_iter = 0
    max_delta = -1
    counts = Count(sample)
    heights = Heights(counts, n)
    
    for x in heights:
        f = func(x, args)
        max_delta = max(max_delta, max(abs(f - heights[x]), abs(f - h_last_iter)))
        h_last_iter = heights[x]
        
    return max_delta * np.sqrt(n)

### p-value для Колмогорова

In [6]:
def Linear(x, args):
    a, b = args
    return (x - a) / (b - a)
def KolmogorovCd(x,n):
    s = 0
    for i in range(1, n):
        s += (-1) ** i * np.exp(-2 * i * i * x * x)
    return 1 + 2 * s
n = 100
kolmog_delta = KolmogorovDelta((0, 9), sample, n, Linear)
p_value2=1-KolmogorovCd(kolmog_delta,n)
p_value2

0.032851885438597406

## b)Проверка c помощью χ²

### Минимизация

In [17]:

def get_norm_pdf(x_cur, teta1_cur, teta2_cur):
    return sps.norm.pdf(x = x_cur, loc = teta1_cur, scale = teta2_cur)


def optim_log(teta_cur, pairs_cur):
    L = 1
    for i in range(len(pairs_cur[0])):
        integral_value = integr.quad(get_norm_pdf, pairs_cur[0][i], pairs_cur[1][i], args = (teta_cur[0], teta_cur[1]))
        L *= integral_value[0]
        
    return -np.log(L)

pairs = np.array([[[-np.inf]*5 + [1]*8 + [2]*6 + [3]*12 + [4]*14 + [5]*18 + [6]*11 + [7]*6 + [8]*13 + [9]*7] , 
                  [[1]*5 + [2]*8 + [3]*6 + [4]*12 + [5]*14 + [6]*18 + [7]*11 + [8]*6 + [9]*13 + [np.inf]*7]])
pairs = np.squeeze(pairs, axis=1)

print("/1/", n)
print("/2/", get_norm_pdf(-np.inf, 4, 3))
print("/3/", integr.quad(get_norm_pdf, -np.inf, 0, args = (4, 3))[0])
    
# Максимизируем логарифм ( минимализируем -lnL)

optima = minimize(fun = optim_log,  x0 = [5, 2], args = (pairs,)) # из графика выборки возьмем 5, 2*sigma (95%) = 4

teta1 = round(optima.x[0], 4)
teta2 = round(optima.x[1], 4)
print("/4/", "teta1, teta2 : ",teta1, teta2)


/1/ 100
/2/ 0.0
/3/ 0.09121121972586788
/4/ teta1, teta2 :  5.2897 2.6795


In [18]:

my_p_full = np.array([integr.quad(get_norm_pdf, pairs[0][i], pairs[1][i], args = (teta1, teta2))[0] for i in range(n)])
my_np_full = my_p_full * n

my_np_res = np.delete(my_np_full, np.where(np.diff(my_np_full) == 0)[0] + 1)
np_res = np.round(my_np_res, 3) 
print("/5/", my_np_res)


delta = np.sum((nums - np_res)**2 / np_res)
print("/6/", "delta = ", delta)
p_value = Chi2XtoInf(delta, 7)
print("p-value", p_value)

/5/ [ 5.46958787  5.50784693  8.66327066 11.8737082  14.18072312 14.75773227
 13.38290855 10.57523613  7.28175447  8.30723179]
/6/ delta =  9.801418398650844
p-value 0.20010899336247517


## p-value > alpha, где alpha = 0.05  => нет оснований отвергать Ho.

## Проверка по Колмогорову

In [19]:
def emp_func(sample_cur, n_cur):
    xy_pairs = np.vstack(( np.sort(sample_cur), np.arange(0, n_cur) / n_cur )).T
    uniq_elem = xy_pairs[0]
    for pair_cur in xy_pairs:
        if pair_cur[0] == uniq_elem[0]:
            pair_cur[1] = uniq_elem[1]
        else:
            uniq_elem = pair_cur
    return xy_pairs

def norm_func(sample_cur, teta1_cur, teta2_cur):
    x1 = np.sort(sample_cur)
    y1 = sps.norm.cdf(x = x1, loc = teta1_cur, scale = teta2_cur)
    return np.vstack((x1, y1)).T

In [21]:
xn = np.array([0]*5 + [1]*8 + [2]*6 + [3]*12 + [4]*14 + [5]*18 + [6]*11 + [7]*6 + [8]*13 + [9]*7)
n = xn.size
sqrt_n = n ** 0.5
teta1 = np.mean(xn)
teta2 = np.std(xn)
print("/1/", n, teta1, teta2)

empf = emp_func(xn, n) 
print("/2/", empf.shape)
print("/3/", empf[:10])

normf = norm_func(xn, teta1, teta2) 
print("/4/", normf.shape)
print("/5/", normf[:10])

delta = sqrt_n * max(np.max(np.abs(empf[:, 1] - normf[:, 1])), np.max(np.abs(empf[1:, 1] - normf[:-1, 1])), np.abs(1 - normf[-1, 1])) # ~delta
print("/6/", np.abs(empf[:, 1] - normf[:, 1])[:10], np.abs(empf[1:, 1] - normf[:-1, 1])[:10], np.abs(1 - normf[-1, 1]))
print("/7/", "delta = ", delta)

/1/ 100 4.77 2.505414137423193
/2/ (100, 2)
/3/ [[0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [1.   0.05]
 [1.   0.05]
 [1.   0.05]
 [1.   0.05]
 [1.   0.05]]
/4/ (100, 2)
/5/ [[0.         0.02846311]
 [0.         0.02846311]
 [0.         0.02846311]
 [0.         0.02846311]
 [0.         0.02846311]
 [1.         0.06619531]
 [1.         0.06619531]
 [1.         0.06619531]
 [1.         0.06619531]
 [1.         0.06619531]]
/6/ [0.02846311 0.02846311 0.02846311 0.02846311 0.02846311 0.01619531
 0.01619531 0.01619531 0.01619531 0.01619531] [0.02846311 0.02846311 0.02846311 0.02846311 0.02153689 0.01619531
 0.01619531 0.01619531 0.01619531 0.01619531] 0.045672642243731576
/7/ delta =  1.013371112422481


In [23]:

N = 50000
x_boot = np.array([(np.round(sps.norm.rvs(size = n, loc = teta1, scale = teta2)))%10 for i in range (N)])
print("/8/", x_boot.shape)

teta1_boot = np.mean(x_boot, axis=1)
teta2_boot = np.std(x_boot, axis=1)
print("/9/", teta1_boot.shape)
print("/10/", teta1_boot[:5])
print("/11/", teta2_boot[:5])

empf_boot = np.apply_along_axis(func1d = emp_func, axis = 1, arr = x_boot, n_cur = n) 
print("/12/", empf_boot.shape)
print("/13/", empf_boot[0][:10])


delta_boot = np.array([])
for i in range(N):
    empf_boot_cur = empf_boot[i]
    normf_boot_cur = norm_func(x_boot[i], teta1_boot[i], teta2_boot[i]) 
    delta_boot_cur = sqrt_n * max(np.max(np.abs(empf_boot_cur[:, 1] - normf_boot_cur[:, 1])), 
                                  np.max(np.abs(empf_boot_cur[1:, 1] - normf_boot_cur[:-1, 1])), 
                                  np.abs(1 - normf_boot_cur[-1, 1])) 
    delta_boot = np.append(delta_boot, delta_boot_cur)



delta_boot = np.sort(delta_boot) 
print("/14/", delta_boot.shape)

l = len([idelta for idelta in delta_boot if idelta >= delta])
print("/15/", l)

p_value = l / N
print("/16/", p_value)
print("p-value = ", p_value)


/8/ (50000, 100)
/9/ (50000,)
/10/ [4.76 4.68 5.14 5.02 4.66]
/11/ [2.27648852 2.20399637 2.40840196 2.0541665  2.21006787]
/12/ (50000, 100, 2)
/13/ [[0.   0.  ]
 [0.   0.  ]
 [0.   0.  ]
 [1.   0.03]
 [1.   0.03]
 [1.   0.03]
 [1.   0.03]
 [1.   0.03]
 [1.   0.03]
 [1.   0.03]]
/14/ (50000,)
/15/ 43364
/16/ 0.86728
p-value =  0.86728


## p-value > alpha, где alpha = 0.05  => нет оснований отвергать Ho.